In [1]:
import ee
import geemap
import importlib

In [2]:
Map = geemap.Map()

In [3]:
ee.Initialize()

In [4]:
import utils.ee_utils as ee_utils

In [5]:
# Reload the module
importlib.reload(ee_utils)

<module 'utils.ee_utils' from 'd:\\python\\CropMapping\\dataset\\utils\\ee_utils.py'>

In [6]:

# Define area of interest.
# roi = ee.Geometry.Polygon(
#                                 [[[1.2897224677359675, 48.38400739957012],
#                                 [1.2897224677359675, 47.93697958008443],
#                                 [3.2068367255484675, 47.93697958008443],
#                                 [3.2068367255484675, 48.38400739957012]]])
roi = ee.Geometry.Polygon( [[[2.220233, 48.116767],
                             [2.301807, 48.116767],
                             [2.301807, 48.167835],
                             [2.220233, 48.167835],
                             [2.220233, 48.116767]]])


# Call function with defined start and end dates and area of interest.
cloudless_collection = ee_utils.create_s2_collection(roi, '2019-05-01', '2019-06-30')

# Define visualization parameters.
visualization = {
        'min': 0.0,
        'max': 0.3,
        'bands': ['B4', 'B3', 'B2'],
}

# Add layer to map.
Map.addLayer(cloudless_collection.median().clip(roi), visualization, 'Cloudless Collection')
Map.centerObject(roi, 10)

In [7]:
s1_collection = ee_utils.create_s1_collection(roi, '2019-05-01', '2019-06-30')


In [8]:
s1_collection.size().getInfo()

14

In [9]:
ee_utils.sen1_print(s1_collection)

,Property,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,system:time_start,2019-05-04,2019-05-16,2019-05-28,2019-06-09,2019-06-21,2019-05-03,2019-05-10,2019-05-15,2019-05-22,2019-06-03,2019-06-08,2019-06-15,2019-06-20,2019-06-27
1,orbitProperties_pass,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING,ASCENDING
2,resolution,H,H,H,H,H,H,H,H,H,H,H,H,H,H
3,resolution_meters,10,10,10,10,10,10,10,10,10,10,10,10,10,10
4,platform_number,A,A,A,A,A,B,B,B,B,B,B,B,B,B
5,productType,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD,GRD
6,orbitNumber_start,27081,27256,27431,27606,27781,16083,16185,16258,16360,16535,16608,16710,16783,16885
7,orbitNumber_stop,27081,27256,27431,27606,27781,16083,16185,16258,16360,16535,16608,16710,16783,16885
8,transmitterReceiverPolarisation,"['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']","['VV', 'VH']"
9,system:band_names,"['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']","['VV', 'VH', 'angle']"


In [10]:
s1_mosaic = ee_utils.get_combined_s1_image(roi, '2019-05-01', '2019-06-30')
s1_mosaic = ee_utils.sen1_add_ratio(s1_mosaic, vv_name='VV', vh_name='VH', ratio_name='ratio')

false_color = {
    'bands': ['VV', 'VH', 'ratio'],
    'min': [-25, -35, 0],
    'max': [0, -5, 10]
}
Map.addLayer(s1_mosaic, false_color, 'S1 Mosaic')
Map.centerObject(roi, 10)

In [21]:
crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
Map.addLayer(crop_map, {}, 'Crop Map')

In [12]:
s1_mosaic

In [13]:
Map

Map(center=[48.14230398135283, 2.2610199999944824], controls=(WidgetControl(options=['position', 'transparent_…

In [14]:
# get roi from polygon geemap
# roi = Map.draw_last_feature.geometry()
# print(roi)

In [15]:
print(roi.getInfo())

{'type': 'Polygon', 'coordinates': [[[2.220233, 48.116767], [2.301807, 48.116767], [2.301807, 48.167835], [2.220233, 48.167835], [2.220233, 48.116767]]]}


In [16]:
sen1_short_print = lambda s2_collection: ee_utils.ee_property_printer(s2_collection, propertie_name_list=['system:time_start','platform_number',
                                                                                           'system:band_names','instrumentMode',                         
                                                                                            'relativeOrbitNumber_stop','relativeOrbitNumber_start',
                                                                                            'cycleNumber'],
                                                             first_is_t_in_millis=True, df_instead_of_print=False)

In [73]:
def download_s1_s2(roi, start_date, end_date, s1_folder="", s2_folder="",
                   s1_file_name="s1_56-48.tif", s2_file_name="s2_56-48.tif",
                   scale=10, bands=['B2', 'B3', 'B4'], proj='EPSG:3857'):
    s1_img = ee_utils.get_combined_s1_image(roi, start_date, end_date)
    s2_img = ee_utils.get_combined_s2_image(roi, start_date, end_date)
    s2_img = s2_img.select(bands)

    # EPSG:4326 or EPSG:3857
    geemap.download_ee_image(s1_img, s1_folder + s1_file_name, crs=proj, scale=scale, region=roi)
    geemap.download_ee_image(s2_img, s2_folder + s2_file_name, crs=proj, scale=scale, region=roi)

In [65]:
download_s1_s2(roi, '2019-05-01', '2019-06-30', scale=10, bands=['B2','B3','B4'])

s1_56-48.tif: |          | 0.00/12.4M (raw) [  0.0%] in 00:00 (eta:     ?)

s2_56-48.tif: |          | 0.00/9.30M (raw) [  0.0%] in 00:00 (eta:     ?)

In [19]:
crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
geemap.download_ee_image(crop_map, "crop_map"+".tif",crs='EPSG:3857', scale=10,region = roi)

crop_map.tif: |          | 0.00/1.55M (raw) [  0.0%] in 00:00 (eta:     ?)

In [74]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
def get_previous_months(harvest_date):
    # Convert harvest date string to datetime object
    harvest_datetime = datetime.strptime(harvest_date, '%Y-%m-%d')
    
    # Create a list to store the previous months
    previous_months = []
    
    # Loop through the previous six months and add them to the list
    for i in range(1, 7):
        previous_month = ((harvest_datetime - relativedelta(months=i+1)).strftime('%Y-%m-%d'), (harvest_datetime - relativedelta(months=i)).strftime('%Y-%m-%d'))
        
        previous_months.append(previous_month)
        
    previous_months.reverse()
    return previous_months

dummy_harvest_date = '2019-08-01'
previous_months = get_previous_months(dummy_harvest_date)
print(previous_months)

[('2019-01-01', '2019-02-01'), ('2019-02-01', '2019-03-01'), ('2019-03-01', '2019-04-01'), ('2019-04-01', '2019-05-01'), ('2019-05-01', '2019-06-01'), ('2019-06-01', '2019-07-01')]


In [75]:
def name_from_roi(roi):
    center = roi.centroid().getInfo()['coordinates']
    x = str(center[0]).replace('.','-')[0:5]
    y = str(center[1]).replace('.','-')[0:5]
    return x + "_" + y + ".tif"

name_from_roi(roi)

'2-261_48-14.tif'

In [76]:
import os
def check_folder_exists(folder):
    if not os.path.exists(folder):
        print("Creating folder: " + folder)
        os.makedirs(folder)

In [77]:
def ts_downloader(roi, harvest_date, scale=10, bands=['B2','B3','B4'], proj='EPSG:3857', dataset_folder="ts_dataset"):
    ts_dates = get_previous_months(harvest_date)
    name = name_from_roi(roi)
    for i, ts_date in enumerate(ts_dates):
        print(f"Downloading Date {i+1}: {ts_date[0]} to {ts_date[1]}")
        check_folder_exists(dataset_folder + "//s1//" + str(i+1))
        check_folder_exists(dataset_folder + "//s2//" + str(i+1))
        download_s1_s2(roi, ts_date[0], ts_date[1], s1_folder="ts_dataset//s1//" + str(i+1) + "//",
                                                    s2_folder="ts_dataset//s2//" + str(i+1) + "//",
                                                    s1_file_name=name,  s2_file_name=name,
                                                    scale=scale, bands=bands)
    
    print("Downloading Crop Map...")
    crop_map = ee.Image('JRC/D5/EUCROPMAP/V1/2018').clip(roi)
    # none vlue to 0
    crop_map = crop_map.unmask(0)
    check_folder_exists("ts_dataset//crop_map")
    geemap.download_ee_image(crop_map, dataset_folder + "//" + "crop_map//" + name, crs='EPSG:3857', scale=10,region = roi)
    
    print("Done!")

In [78]:
ts_downloader(roi, dummy_harvest_date, scale=10, bands=['B2','B3','B4'])

Creating folder: ts_dataset//s1//1
Creating folder: ts_dataset//s2//1


2-261_48-14.tif: |          | 0.00/12.4M (raw) [  0.0%] in 00:00 (eta:     ?)

2-261_48-14.tif: |          | 0.00/9.30M (raw) [  0.0%] in 00:00 (eta:     ?)

Creating folder: ts_dataset//s1//2
Creating folder: ts_dataset//s2//2


2-261_48-14.tif: |          | 0.00/12.4M (raw) [  0.0%] in 00:00 (eta:     ?)

...

2-261_48-14.tif: |          | 0.00/12.4M (raw) [  0.0%] in 00:00 (eta:     ?)

2-261_48-14.tif: |          | 0.00/9.30M (raw) [  0.0%] in 00:00 (eta:     ?)

Creating folder: ts_dataset//crop_map


2-261_48-14.tif: |          | 0.00/1.55M (raw) [  0.0%] in 00:00 (eta:     ?)

Done!
